In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Callable

import jax.numpy as jnp
import numpy as np
import torchvision
from jax import random
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

from bayesian_active_learning.acquisition_functions import (
    get_acquisition_function,
)
from bayesian_active_learning.experiment import experiment_run
from bayesian_active_learning.utils import one_hot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

# Summary

- The following notebook attempts to verify that the BALD acquisition function outperforms random sampling of the pool set, when the pool set has been artificially imbalanced

- The original pool set is more or less balanced, since the original MNIST training dataset is well balanced.
- However, we artifically introduce class imbalance to the pool set by removing 75% of the 2s present.
- Since the BALD score can actively "target" the smaller amount of twos in acquisition, we expected it to perform better than random sampling, where we have to "get lucky" to come across a two in the acquired points

- The notebook produces a figure in `figures/03_modified_pool_set`.
- The figure shows that when using a small batch size, using the BALD score outperforms random sampling from the pool, which is closer to the plot shown in the original paper

# 1. Experimental setup

In [ ]:
def full_experiment(acquisition_function_name: str, seed: int) -> jnp.ndarray:
    # get acquisition function
    acquisition_function = get_acquisition_function(acquisition_function_name)

    # grab all datasets + preprocess

    full_train_dataset = torchvision.datasets.MNIST(
        "../datasets", train=True, download=True
    )
    full_test_dataset = torchvision.datasets.MNIST(
        "../datasets", train=False, download=True
    )

    num_classes = len(full_train_dataset.classes)
    total_train_samples = len(full_train_dataset.data)
    total_test_samples = len(full_train_dataset.data)

    all_train_X = np.array(full_train_dataset.data) / 255.0
    all_train_y = one_hot(np.array(full_train_dataset.targets), k=num_classes)

    all_test_X = np.array(full_test_dataset.data) / 255.0
    all_test_y = one_hot(np.array(full_test_dataset.targets), k=num_classes)

    # shuffle the datasets according to the seed and split into initial train, test and pool sets
    num_initial_train_points = 100
    num_validation_points = 100

    rng = np.random.default_rng(seed)
    shuffle_idx = rng.permutation(len(all_train_X))

    all_train_X, all_train_y = all_train_X[shuffle_idx], all_train_y[shuffle_idx]

    initial_train_X, val_X, initial_pool_X = np.split(
        all_train_X,
        [num_initial_train_points, num_initial_train_points + num_validation_points],
    )
    initial_train_y, val_y, initial_pool_y = np.split(
        all_train_y,
        [num_initial_train_points, num_initial_train_points + num_validation_points],
    )

    # perform class based subsampling of pool set
    key = random.PRNGKey(seed)

    two_label = jnp.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
    two_mask = (initial_pool_y @ two_label).astype(bool)

    pool_twos = initial_pool_X[two_mask]
    num_twos = len(pool_twos)

    pool_X_removed_twos = initial_pool_X[~two_mask]
    pool_y_removed_twos = initial_pool_y[~two_mask]

    # subsample a proportion of the pool of twos and labels
    key = random.PRNGKey(0)
    key, subkey = random.split(key, 2)
    subsampling_idxs = random.choice(
        subkey, np.arange(num_twos), (int(num_twos * 0.25),), replace=False
    )

    subsampled_pool_twos = pool_twos[subsampling_idxs]
    subsampled_pool_twos_labels = np.stack([two_label] * num_twos)

    # reintroduce into pool
    new_pool_X = np.concatenate([pool_X_removed_twos, subsampled_pool_twos])
    new_pool_y = np.concatenate([pool_y_removed_twos, subsampled_pool_twos_labels])

    # shuffle pool
    key, subkey = random.split(key, 2)
    shuffle_idx = random.permutation(subkey, np.arange(len(new_pool_X)))

    new_pool_X = new_pool_X[shuffle_idx]
    new_pool_y = new_pool_y[shuffle_idx]

    # run the active learning procedure with the acquisition function and datasets
    test_accuracy_history = experiment_run(
        train_set=(initial_train_X, initial_train_y),
        val_set=(val_X, val_y),
        pool_set=(new_pool_X, new_pool_y),
        test_set=(all_test_X, all_test_y),
        weight_decay=1e-2,
        acquisition_fn=acquisition_function,
        num_predictive_samples=100,
        num_acquired_points_per_iteration=100,
        num_iterations=5,
        key=key,
    )

    return test_accuracy_history

# Setup parameter grid

In [ ]:
acquisition_functions = ["BALD", "Random"]
seeds = np.arange(6)

param_grid = {"acquisition_function_name": acquisition_functions, "seed": seeds}
arg_list = list(ParameterGrid(param_grid))

In [ ]:
results = []

for args in tqdm(arg_list):
    results.append(full_experiment(**args))

# Plotting

In [ ]:
grouped_results = {af: [] for af in acquisition_functions}

for result, arg in zip(results, arg_list):
    grouped_results[arg["acquisition_function_name"]].append(result)

grouped_results = {af: jnp.stack(r) for af, r in grouped_results.items()}

In [ ]:
for label, results in grouped_results.items():
    median = 100 * np.median(results, axis=0)
    lower_quartile = 100 * np.percentile(results, 25, axis=0)
    upper_quartile = 100 * np.percentile(results, 75, axis=0)

    plt.plot(100 * np.arange(len(median)), median, label=label)
    plt.fill_between(
        100 * np.arange(len(median)), lower_quartile, upper_quartile, alpha=0.25
    )

plt.xlabel("Number of acquired points")
plt.ylabel("Test accuracy")
plt.ylim(75, 100)
plt.legend()
plt.savefig("../figures/03_modified_pool_set/plot.png", bbox_inches="tight")

plt.show()